# Introduction

The data used for this Exploratory Data Analysis comes from two sources. The first is a database in SQLite3 called `flow.db` that was generated by a CLI script named `flowmodoro.py` I used to track the time spent focusing during my study sessions. An important caveat about this data is that the time is spent strictly focusing without pauses. The other source is a CSV file extracted from Anki using the AnkiConnect add-on and the Python script `anki_info.py` to extract the review instances of the cards in the decks dedicated to the exam.

The data evaluated for this analysis corresponds to the logs between June 16, 2024, and January 25, 2025 (the day of the exam). Furthermore, starting September 9, 2024, all time logged in `flow.db` can be assumed to have been logged while studying in the public library, excluding local holidays like September 11, 2024, weekends and Christmas break ranging from December 24, 2024, and January 7, 2025, whereas prior to that, I was studying at home
## `flow.db` Data Structure

The `flow.db` data is structured as follows, with three tables: `daily_log`, `projects`, and `break_level`.

-   `daily_log` contains the data referring to each focus cycle entry and has the following columns:

| Column        | DataType   | Description                                                                                                       |
|---------------|------------|-------------------------------------------------------------------------------------------------------------------|
| id            | int        | Primary key, used to identify the log entry                                                                      |
| project_id    | int        | Foreign key, used to relate to a project                                                                         |
| started       | str        | Date in YYYY-MM-DD HH:MM:SS+ZZ:ZZ format corresponding to the start datetime. This column has an Index.          |
| ended         | str        | Date in YYYY-MM-DD HH:MM:SS+ZZ:ZZ format corresponding to the end datetime                                         |
| mins_w        | int        | Number of minutes worked in a focus cycle                                                                         |
| accomp        | str        | Description of specific tasks worked on                                                                            |

The data in the `accomplished` column follows a specific structure. It usually begins with the primary activity performed during that cycle. The content of the `accomplished` column, depending on its first word, can be interpreted as follows:

As part of the Bear Hunter System, which I use for learning or reviewing information:

-   "AIM", "SHOOT", or "SKIN".
-   "PRESTUDY".

As part of Active Recall:

-   "BD" or "BRAIN" signifies Brain Dump, a component of active recall.
-   "SIR" stands for Spacing, Interleaving, and Retrieval, another aspect of active recall.
-   "EXAM", "FIR", "GOBIR", or "SIMULACRO" signifies time spent taking practice exams.

As part of New Anki Cards or Anki Managment:

-   "ANKI" indicates time spent creating new Anki cards.
-   ANKI can be followed by "ACM", that denotes Active Card Management, the time dedicated to managing Anki cards.

As part of watching lectures:

-   "VIDEO" or "CLASE" corresponds to watching an academy lecture.

As part of Anki Reviews of the deck `02-EXAM_Qs`:

-   "OFICIALES" time spent doing Anki reviews of `02-EXAM_Qs` deck.

Empty strings or initial words in the `accomplished` column that do not adhere to this structure are to be expected.

Finally for this analysis, the data in `projects` and `break_level` is not relevant because projects while it houses the project name, in this instance I already know that the project `BIR` corresponding to the data for this analysis is project_id = 2, and break_level is used internaly by the `flowmodoro.py` script.

For more info in `flowmodoro.py` script or `flow.db`, visit the [GitHub repository](https://github.com/Oddman43/Flowmodoro)

## `anki_info.csv` Data Structure

### Anki Information

Anki is a popular open-source flashcard app and a great tool for learners. While studying for the exam, I primarily used two decks: the first, named `01-BIR`, housed the flashcards I created while studying, and the second, named `02-EXAM_Qs`, contained the actual questions asked in the last 20 years of exams, plus relevant questions from the Chemistry, Pharmacy, and Medicine exams.

The cards in the `01-BIR` deck could will have the following tags associated

-   `00Daily::` followed by `numMonth::` and `YYYYMMDD` used to suspend the cards the same day.
-   A string of the overview topic of the tag, like `DNA` or `Nervioso`.

Cards with no tags can be expected

The cards in the `02-EXAM_Qs` deck are identified by a tag corresponding to the exam, using the three-letter abbreviation (`BIR`, `QIR`, `FIR`, `MIR`) followed by `::` and the year the question was asked. Finally, this deck also contains cards with the tag `GOBIR`, which are image-occlusion cards made from pictures of the practice questions provided by an academy.

In Anki, a Note is the source of information, like a template containing fields such as a question and its answer, while a Card is a specific question-and-answer pair generated from a Note. A single Note can generate multiple Cards, each testing a different aspect of the information. For this Exploratory Data Analysis, I will use Cards instead of Notes because this approach provides more granularity for Notes that have multiple associated Cards, allowing for a more detailed analysis of review performance.

### `anki_info.py`

The `anki_info.py` script connects with Anki using the code in `anki_connect.py` and grabs all the data regarding the review history of the notes in the decks `01-BIR` and `02-EXAM_Qs` and outputs a DataFrame.

The `anki_info.csv` data has the following columns:

| Column    | DataType | Description                                                                                                                               |
|-----------|----------|-------------------------------------------------------------------------------------------------------------------------------------------|
| note_id   | int      | Unique id for each note, derived from the UNIX timestamp in milliseconds of the creation moment                                           |
| card_id   | int      | Unique id for each card, derived from the UNIX timestamp in milliseconds of the creation moment                                           |
| tags      | str      | String representation of the list of tags associated with each note, as described in the Anki Information section above                   |
| reviews   | str      | Contains a history of each review event for the card, stored as a list of dictionaries (explained in the next table)                     |

For each review the data has the following columns:

| Column      | DataType | Description                                                 |
|-------------|----------|-------------------------------------------------------------|
| unix_time   | int      | UNIX timestamp in ms of the review instance                |
| pressed     | int      | Integer corresponding to the button pressed; being 1 = Again, 2 = Hard, 3 = Good, 4 = Easy|
| review_offset | int      | Time spent reviewing the card in ms                       |

On pressed column it can also be "0", meaning a reschedule of the card, review instances with this pressed need to be filtered out.

## Objectives

The main questions to ask with this Exploratory Data Analysis are:

-   Analyze whether the time invested in creating anki cards is recovered in the long run by doing reviews and if so at what point this happens
-   Which anki cards have cost me the most, both in terms of time spent and success rate? Does the deck influence it?
-   Is there significant difference in study time between studying at home vs. the library?
-   Investigate potential causes for observed peaks in study time, such as proximity to exams, changes in study strategies, or external factors.

# Data Loading and Cleaning

## Used libraries

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import json
import urllib.request
from dataclasses import dataclass
import sqlite3
import os
import logging
logging.basicConfig(
    level= logging.INFO,
    format= "%(asctime)s - %(levelname)s - %(message)s"
)

## Loading data from Anki

### Anki Connect script

Copied from [Anki Connect documentation](https://git.sr.ht/~foosoft/anki-connect/)

In [ ]:
def request(action: str, **params) -> dict:
    return {"action": action, "params": params, "version": 6}


def invoke(action: str, **params):
    requestJson = json.dumps(request(action, **params)).encode("utf-8")
    response = json.load(
        urllib.request.urlopen(
            urllib.request.Request("http://127.0.0.1:8765", requestJson)
        )
    )
    if len(response) != 2:
        raise Exception("response has an unexpected number of fields")
    if "error" not in response:
        raise Exception("response is missing required error field")
    if "result" not in response:
        raise Exception("response is missing required result field")
    if response["error"] is not None:
        raise Exception(response["error"])
    return response["result"]

### Script to get card info

#### Anki Card Object

In [2]:
@dataclass
class Anki_Card:
    """
    AnkiCard Object
    """
    nid: int
    cid: int
    tags: list
    reviews: list[dict]

    def __post_init__(self) -> None:
        """Runs function post init
        """
        self._clean_reviews()

    def _clean_reviews(self):
        """
        For each card takes the apropiate fields
        and stores them in a list of dicts
        """
        clean: list = []
        for rev in self.reviews:
            clean_dict = {
                "unix_time": rev["id"],
                "pressed": rev["ease"],
                "time_review_offset": rev["time"],
            }
            clean.append(clean_dict)
        self.reviews = clean
        return

    def format_save(self):
        """Formats the object as dict

        Returns:
            dict: contains object info
        """
        return {
            "note_id": self.nid,
            "card_id": self.cid,
            "tags_list": self.tags,
            "reviews": self.reviews,
        }

#### Extracting data from Anki

The invoke function is used to acces the cards info in the apropiate decks

In [ ]:
decks: list[str] = ["01-BIR", "02-EXAM_Qs"]
cards: list = []

for deck in decks:
    info_reviews: dict = {
        k: v
        for k, v in invoke(
            "getReviewsOfCards", cards=(invoke("findCards", query=f"deck:{deck}"))
        ).items()
        if len(v) >= 1
    }
    cards_info: list[dict] = invoke(
        "cardsInfo", cards=[int(cid) for cid, _ in info_reviews.items()]
    )
    for card_dic in cards_info:
        card = Anki_Card(
            card_dic["note"],
            card_dic["cardId"],
            invoke("getNoteTags", note=card_dic["note"]),
            info_reviews[str(card_dic["cardId"])],
        )
        cards.append(card)
    logging.info(f"Extraceted infor from {deck}")

Load the card data into a DataFrame and save as parquet

In [ ]:
raw_parquet_path: str = "../data/raw_anki.parquet"
raw_cards_df: pd.DataFrame = pd.DataFrame([card.format_save() for card in cards])
raw_cards_df.to_parquet(raw_parquet_path, index=False)
logging.info(f"Saved data into {raw_parquet_path}")

Validate that the file exists and head of the DataFrame

### Data Verification from Anki
#### Missing Values

In [2]:
raw_cards_df = pd.read_parquet("../data/raw_anki.parquet")
if raw_cards_df.isnull().sum().sum() != 0:
    logging.info("Data Frame raw_cards_df has null values")
logging.info("No null values in raw_cads")

2025-11-18 12:59:38,486 - INFO - No null values in raw_cads


Empty tags in `tags_list`

In [3]:
raw_cards_df["not_empty_tags"] = raw_cards_df["tags_list"].apply(lambda x: len(x) > 0)
if raw_cards_df["not_empty_tags"].value_counts().get(False, 0) > 0:
    logging.info(f"Missing tags, will be assigned 'BIR' tag")
    missing_tags = raw_cards_df["not_empty_tags"] == False
    raw_cards_df.loc[missing_tags, "tags_list"] = raw_cards_df.loc[missing_tags, "tags_list"].apply(lambda _: ["00Daily"])
else:
    logging.info("No missing tags")
raw_cards_df = raw_cards_df.drop(columns= ["not_empty_tags"])

2025-11-18 13:00:09,749 - INFO - Missing tags, will be assigned 'BIR' tag


### Data Transformation
#### Adding Deck column

In [4]:
deck_tags: set = {"BIR", "MIR", "QIR", "FIR", "GOBIR", "Simulacro"}
raw_cards_df["deck"] = raw_cards_df["tags_list"].apply(
    lambda tags_list: 
    "02_exam" if any(any(deck_tag in tag for deck_tag in deck_tags) for tag in tags_list)
    else "01_own"
)
logging.info("Created deck column from tag info")

2025-11-18 13:00:47,179 - INFO - Created deck column from tag info


#### Unnesting reviews column

In [5]:
raw_cards_df_2: pd.DataFrame = raw_cards_df.explode("reviews")
logging.info("Unnested reviews column")
raw_cards_df_2 = pd.concat(
    [
        raw_cards_df_2.drop(columns=["reviews"]).reset_index(drop=True),
        pd.json_normalize(raw_cards_df_2["reviews"]).reset_index(drop=True) # type: ignore
    ],
    axis=1
)
logging.info("Json normalized the reviews column")

2025-11-18 13:01:42,601 - INFO - Unnested reviews column
2025-11-18 13:01:42,941 - INFO - Json normalized the reviews column


Changing the review time to seconds and unix timestamp to datetime object plus renaming the columns


In [6]:
raw_cards_df_2["time_review_offset"] = raw_cards_df_2["time_review_offset"] / 1000
raw_cards_df_2["unix_time"] = pd.to_datetime(raw_cards_df_2["unix_time"], unit = "ms")
raw_cards_df_2 = raw_cards_df_2.rename(columns= {"time_review_offset": "review_time_seconds", "unix_time": "date_reviewed"})

Adding date_day column for grouping later, and filtering by date with the cut off 2025-01-26

In [ ]:
raw_cards_df_2["date_day"] = raw_cards_df_2["date_reviewed"].dt.date
date_cut_off: pd.Timestamp = pd.to_datetime("2025-01-26")
raw_cards_df_2 = raw_cards_df_2[raw_cards_df_2["date_reviewed"] <= date_cut_off]

Saving the clean data into parquet file

In [ ]:
clean_cards_df: pd.DataFrame = raw_cards_df_2
logging.info("Renamed columns and changed time_reviews into seconds and unix_time to datateime")
clean_cards_df.info()

In [ ]:
clean_parquet_path_anki: str = "../data/clean_anki.parquet"
clean_cards_df.to_parquet(clean_parquet_path_anki)
if not os.path.exists(clean_parquet_path_anki):
    logging.error(f"Parquet file in {clean_parquet_path_anki} does not exist")
    raise FileExistsError("Parquet file was not created")
else:
    logging.info(f"Saved clean data in {clean_parquet_path_anki}")

## Loading data from Flowmodoro

### Extraction from SQLite3

In [ ]:
db_path: str = "../data/raw_flow.db"
with sqlite3.connect(db_path) as db:
    cur: sqlite3.Cursor = db.cursor()
    query: str = "SELECT started, mins_worked, accomplished FROM daily_log WHERE project_id = 2;"
    result = cur.execute(query)
    raw_flowmodoro_df: pd.DataFrame = pd.DataFrame(cur.fetchall(), columns= ["started", "mins_worked", "accomplished"]) 

raw_flowmodoro_df.info()

### Data verification for Flowmodoro data
#### Missing values

In [ ]:
if raw_flowmodoro_df.isnull().sum().sum() != 0:
    logging.info("Data Frame raw_cards_df has null values")
logging.info("No null values in raw_cads")

### Data Transformation

In [ ]:
raw_flowmodoro_df["started"] = pd.to_datetime(raw_flowmodoro_df["started"])

To represent the principal activity done in a cycle, the group describen in the Introduction with the following column names:

- Bear Hunter System denoted as `bhs`
- Time spent creating new Anki cards as `anki_craft`
- Time spent doing active recall as `active_recall`
- Time spent watching lectures as `lectures`
- Time spent doing Anki reviews of the deck 02_EXAM_Qs as `anki_exam`

In [ ]:
raw_flowmodoro_df["working_on"] = raw_flowmodoro_df["accomplished"].str.split().str[0]
subs_dict: dict = {
    "AIM": "bhs",
    "SHOOT": "bhs",
    "SKIN": "bhs",
    "PRESTUDY": "bhs",
    "BD": "active_recall",
    "SIR": "active_recall",
    "EXAM": "active_recall",
    "FIR": "active_recall",
    "GOBIR": "active_recall",
    "SIMULACRO": "active_recall",
    "BRAIN": "active_recall",
    "ANKI": "anki_craft",
    "VIDEO": "lectures",
    "CLASE": "lectures",
    "OFICIALES": "anki_exam"
}
raw_flowmodoro_df["working_on"] = raw_flowmodoro_df["working_on"].str.upper().map(subs_dict)

Now lets check for missing values in working_on and check for missed spellings

In [ ]:
print(
    raw_flowmodoro_df.loc[
        raw_flowmodoro_df[["accomplished", "working_on"]].isnull().any(axis=1),
        ["accomplished", "working_on"]
    ]
)

The following ids correspond to the working_on:

- bhs: 0, 1, 2, 3, 4, 6, 7, 12, 113, 224, 273, 385, 594, 961
- active_recall: 413, 758, 977
- anki_exam: 412, 754
- admin: 93, 103, 142, 219, 401, 407

In [ ]:
raw_flowmodoro_df.loc[
    [0, 1, 2, 3, 4, 6, 7, 12, 113, 224, 273, 385, 594, 961],
    "working_on"
] = "bhs"
raw_flowmodoro_df.loc[
    [413, 758, 977],
    "working_on"
] = "active_recall"
raw_flowmodoro_df.loc[
    [412, 754],
    "working_on"
] = "anki_exam"
raw_flowmodoro_df.loc[
    [93, 103, 142, 219, 401, 407],
    "working_on"
] = "admin"

clean_flowmodoro_df: pd.DataFrame = raw_flowmodoro_df

Saving the clean data into a parquet file

In [ ]:
clean_parquet_path_flowmodoro: str = "../data/clean_flowmodoro.parquet"
clean_flowmodoro_df.to_parquet(clean_parquet_path_flowmodoro)
if not os.path.exists(clean_parquet_path_flowmodoro):
    logging.error(f"Parquet file in {clean_parquet_path_flowmodoro} does not exist")
    raise FileExistsError("Parquet file was not created")
else:
    logging.info(f"Saved clean data in {clean_parquet_path_flowmodoro}")

## Creating a merged DataFrame
Reading from clean data parquet files

In [ ]:
flowmodoro: pd.DataFrame = pd.read_parquet("../data/clean_flowmodoro.parquet")
anki: pd.DataFrame = pd.read_parquet("../data/clean_anki.parquet")
logging.info("Loaded clean data parquet files")

Grouping anki info by date and deck

In [ ]:
anki_grouped: pd.DataFrame = anki.groupby(["date_day", "deck"])["review_time_seconds"].sum().reset_index()
anki_grouped["review_time_mins"] = anki_grouped["review_time_seconds"] / 60
anki_grouped = anki_grouped.drop("review_time_seconds", axis= 1)
logging.info("Created anki_gropued")
anki_grouped.info()

Grouping flowmodoro info by date and activity

In [ ]:
flowmodoro["date_day"] = flowmodoro["started"].dt.date
flowmodoro_grouped: pd.DataFrame = flowmodoro.groupby(["date_day", "working_on"])["mins_worked"].sum().reset_index()
logging.info("Created flowmodoro_grouped")
flowmodoro_grouped.info()

Renaming cols so they are the same in both df and concatenating

In [ ]:
anki_grouped.columns = ["date_day", "activity", "mins"]
flowmodoro_grouped.columns = ["date_day", "activity", "mins"]
bir_time: pd.DataFrame = pd.concat([anki_grouped, flowmodoro_grouped], ignore_index= False)
logging.info("Joined Anki and Flowmodoro data")
bir_time.info()

Pivoting wider and filling NA with 0

In [ ]:
bir_time_wide: pd.DataFrame = bir_time.pivot(index= "date_day", columns= "activity", values = "mins").fillna(0)
logging.info("Pivoted wider bir time")
bir_time_wide.info()

Since the time spent doing the 02_exam deck was double counted in anki_exam i will 

In [ ]:
bir_time_wide["exam_anki"] = bir_time_wide[["02_exam", "anki_exam"]].max(axis=1)
bir_time_wide = bir_time_wide.drop(columns=["02_exam", "anki_exam"])

Saving wide df into parquet file

In [ ]:
clean_bir_time_path: str = "../data/clean_bir.parquet"
bir_time_wide.to_parquet(clean_bir_time_path)
if not os.path.exists(clean_bir_time_path):
    logging.error(f"Parquet file in {clean_bir_time_path} does not exist")
    raise FileExistsError("Parquet file was not created")
else:
    logging.info(f"Saved clean data in {clean_bir_time_path}")